In [12]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from explainable_fact_checking.plot.notebook_utility import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
df = xfc.load_preprocess_explanations(experiment_code_list=[
    'fbs_time_1.0',
    'fbs_time_2.0',
])

In [14]:
df

,unit_text,unit_index,SUPPORTS,REFUTES,NOT ENOUGH INFO,num_samples,label,execution_time,id,NOT ENOUGH INFO_predict_proba,...,model_path,explainer_name,dataset_dir,top,perturbation_mode,dataset_file_name,mode,predicted_label,score_on_predicted_label,model_id
5155,"Bill Persky (born September 9, 1931) is an Ame...",0,0.194953,-0.043362,-0.151591,128,SUPPORTS,2.438616,268,0.018809,...,/homes/bussotti/feverous_work/feverousdata/mod...,lime,/home/bussotti/XFCresults/,100,only_evidence,feverous_dev_SO_01.jsonl,NaN,SUPPORTS,0.194953,feverous_verdict_predictor
5156,Persky was born to a [[American_Jews|Jewish]] ...,1,0.281257,-0.025758,-0.255499,128,SUPPORTS,2.438616,268,0.018809,...,/homes/bussotti/feverous_work/feverousdata/mod...,lime,/home/bussotti/XFCresults/,100,only_evidence,feverous_dev_SO_01.jsonl,NaN,SUPPORTS,0.281257,feverous_verdict_predictor
5157,Persky has been married three times.,2,-0.084214,-0.024018,0.108232,128,SUPPORTS,2.438616,268,0.018809,...,/homes/bussotti/feverous_work/feverousdata/mod...,lime,/home/bussotti/XFCresults/,100,only_evidence,feverous_dev_SO_01.jsonl,NaN,SUPPORTS,-0.084214,feverous_verdict_predictor
5158,His third wife is New York advertising executi...,3,0.293867,-0.017703,-0.276164,128,SUPPORTS,2.438616,268,0.018809,...,/homes/bussotti/feverous_work/feverousdata/mod...,lime,/home/bussotti/XFCresults/,100,only_evidence,feverous_dev_SO_01.jsonl,NaN,SUPPORTS,0.293867,feverous_verdict_predictor
5159,Bill Persky (a TV director) was born to a Jewi...,0,-0.201900,0.091101,1.110799,128,SUPPORTS,2.438616,268,0.018809,...,/homes/bussotti/feverous_work/feverousdata/mod...,lime,/home/bussotti/XFCresults/,100,only_evidence,feverous_dev_SO_01.jsonl,NaN,SUPPORTS,-0.201900,feverous_verdict_predictor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47296,Government is Radostin Radev.,2,0.084432,0.065461,-0.149893,32,SUPPORTS,4.170465,51676,0.435392,...,/homes/bussotti/feverous_work/feverousdata/mod...,shap,/home/bussotti/XFCresults/,100,only_evidence,original_TO_01_formatted.jsonl,KernelExplainer,SUPPORTS,0.084432,feverous_verdict_predictor
47297,[[Provinces_of_Bulgaria|Province]] (Oblast) is...,3,-0.020930,-0.008978,0.029908,32,SUPPORTS,4.170465,51676,0.435392,...,/homes/bussotti/feverous_work/feverousdata/mod...,shap,/home/bussotti/XFCresults/,100,only_evidence,original_TO_01_formatted.jsonl,KernelExplainer,SUPPORTS,-0.020930,feverous_verdict_predictor
47298,Kostenets\nКостенец is [[Bulgaria|Bulgaria]] ;...,4,0.251075,-0.101128,-0.149947,32,SUPPORTS,4.170465,51676,0.435392,...,/homes/bussotti/feverous_work/feverousdata/mod...,shap,/home/bussotti/XFCresults/,100,only_evidence,original_TO_01_formatted.jsonl,KernelExplainer,SUPPORTS,0.251075,feverous_verdict_predictor
47299,[[Countries_of_the_world|Country]] is [[Bulgar...,5,0.064073,-0.028684,-0.035389,32,SUPPORTS,4.170465,51676,0.435392,...,/homes/bussotti/feverous_work/feverousdata/mod...,shap,/home/bussotti/XFCresults/,100,only_evidence,original_TO_01_formatted.jsonl,KernelExplainer,SUPPORTS,0.064073,feverous_verdict_predictor


In [15]:
# query explainer_name == 'lime'
df.query('explainer_name == "lime"')['num_samples'].unique()


array([ 128, 4096, 8192,  256, 1024, 2048,   32,   64,  512])

In [16]:
# query explainer_name == 'shap'
shap_df = df.query('explainer_name == "shap"')

In [32]:
# select first element of each id (id_cols are = ['num_samples', 'explainer_name', 'dataset_name', 'id','model_name'])
# plot in log scale y axis = 'execution_time' x axis = 'num_samples'. with mean and std, grouping by id_cols    
# use plotly express
def plot_time_vs_num_samples(df, id_cols, log_scale=True):
    df = df.groupby(id_cols, as_index=False).first()
    # groupby 'id'and compute mean and std of 'execution_time'
    df = df.groupby(list(set(id_cols) - set(['id'])), as_index=False).agg({'execution_time': ['mean', 'std']})
    # merge columns names levels
    df.columns = df.columns.map('_'.join).str.strip('_')
    fig = px.line(df, x='num_samples', y='execution_time_mean', color='explainer_name', symbol='model_name', error_y='execution_time_std', log_y=False)
    return fig
    
fig = plot_time_vs_num_samples(df, id_cols=['num_samples', 'explainer_name', 'dataset_name', 'id','model_name'], log_scale=True)
fig.update_layout(**layout_dict, **h_legend_dict
                  )
fig = end_fig_func(fig)
fig.show()

In [31]:
id_cols=['num_samples', 'explainer_name', 'dataset_name', 'id','model_name']
to_plot = df
to_plot['n_evidence'] = to_plot.groupby(id_cols, as_index=False).transform('size')
id_cols.append('n_evidence')
to_plot = to_plot.groupby(id_cols, as_index=False).first()
# groupby 'id'and compute mean and std of 'execution_time'
to_plot = to_plot.groupby(list(set(id_cols) - set(['id'])), as_index=False).agg({'execution_time': ['mean', 'std']})
# merge columns names levels
to_plot.columns = to_plot.columns.map('_'.join).str.strip('_')
fig = px.line(to_plot, x='n_evidence', y='execution_time_mean', color='num_samples', line_dash='explainer_name', error_y='execution_time_std', log_y=False)
fig.update_layout(**layout_dict, **h_legend_dict
                  )
fig = end_fig_func(fig)
fig.show()